## Definicion del contexto

El contexto con el que quiero trabajar mi tarea es un proyecto personal en el que me encuentro trabajando el cual corresponde a un bot de trading algorítmico basado en señales. Éste toma estrategias de inversion usando indicadores técnicos y genera señales de compra y venta que ejecuta en los mercados a los que se encuentra conectado mediante API rest o recibiendo datos de manera ágil usando websockets de algunos comercios. Antes poner el bot a funcionar es fundamental hacer backtesting para validar la aplicabilidad de las estrategias de inversion. Éstas son validadas a partir de métricas como el ratio de sharpe, max drawdown, drawdown time, entre otras métricas. Para ésta tarea implementaremos algunos patrones que se pueden aplicar para desarrollar una herramienta de backtesting que nos puede servir para validar las estrategias de inversión que va a implementar nuestro bot.

![TradingBot](TradingBot.png)

# Identificación de requerimientos asociados al caso de uso:
Este sistema busca testear estrategias de inversión algorítmica a partir de señales construidas por indicadores financieros.

Haz clic en cada ítem para desplegar los problemas abordados y su solución con patrones de diseño:

<details> <summary>Problema 1</summary>
  
**Problema:** Se desea probar diferentes estrategias de trading (como medias móviles, RSI, MACD), pero sin acoplar el motor de backtesting a cada una.

**Patrón propuesto:** Strategy

**Justificación:** Permite encapsular distintas estrategias dentro de una interfaz común (`TradingStrategy`), lo que hace posible intercambiarlas en tiempo de ejecución y probarlas de manera independiente del motor principal. Esto facilita la extensión del sistema con nuevas estrategias sin modificar el código existente, alineándose con los principios SOLID.
</details>

<details> <summary>Problema 2</summary>
  
**Problema:** Se necesita que múltiples componentes del sistema (gráfico, log de operaciones, alertas, backtester) reaccionen a eventos generados durante los cambios en el mercado, como la ejecución de una orden o activar la ejecucion del `BacktestEngine` para generar las señales de la estrategia.

**Patrón propuesto:** Observer

**Justificación:** Permite que distintos módulos se suscriban al `MarketDataFeed` u otros generadores de eventos, sin acoplarse directamente a ellos. Así, se puede añadir o remover observadores como `Logger`, `ChartUpdater` o `BacktestEngine` sin alterar el flujo principal del sistema.
</details>

<details> <summary>Problema 3</summary>

**Problema:** El proceso de backtesting debe seguir una estructura general compuesta por pasos como carga de datos, limpieza, ejecución de la estrategia, evaluación y generación de reportes. Sin embargo, cada tipo de estrategia o entorno puede requerir personalizar uno o varios de estos pasos sin alterar el flujo general.

**Patrón propuesto:** Template Method

**Justificación:** Se define en una clase base (`BacktestEngine`) un método plantilla (`run`) que establece el flujo general del proceso de backtesting. Las subclases concretas sobrescriben pasos específicos como `load_data` o `execute_strategy`, permitiendo reutilizar la estructura base y mantener la coherencia del flujo. Este patrón favorece la extensibilidad, la reutilización de código y el cumplimiento de los principios SOLID, especialmente el de inversión de dependencias y el abierto/cerrado.

</details>


## Problema 1: **Patron strategy**
El patron strategy resulta ser muy funcional con un Backtest Engine que se encarga de realizar las validaciónes de las estrategias como sharpe ratio y los drawdown ya que nos permite tener un dinamismo en la tipología de la estrategia. Adicionalmente usar la interfaz Trading Strategy, nos permite dar un dinamismo al tipo de estrategia, es decir, si en un futuro no usamos estrategias basadas en indicadores técnicos sino en fundamentales, el diseño nos permite agregarlo con facilidad ya que queda abierto a extensiones lo que involucra el principio Open-Close que nos garantiza el patrón.



![StrategyPattern](StrategyPattern.svg)

In [4]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import List


@dataclass(frozen=True)
class Candlestick:
    """Clase que representa una vela en el gráfico de precios."""
    open: List[float]
    high: List[float]
    low: List[float]
    close: List[float]

@dataclass(frozen=True)
class Signals:
    """Clase que representa las señales generadas por una estrategia de trading."""
    buy: List[int]
    sell: List[int]
    hold: List[int]


# Interfaz
class TradingStrategy(ABC):
    @abstractmethod
    def generate_signals(self):
        pass

# Clase abstracta que implementa la interfaz TradingStrategy
class CandlestickStrategy(TradingStrategy):
    def __init__(self, candlesticks: List[Candlestick] = None):
        self.candlesticks = candlesticks

    @abstractmethod
    def generate_signals(self):
        """Genera señales de trading basadas en los candlesticks."""
        pass

# Estrategias concretas
class MovingAverageStrategy(CandlestickStrategy):
    def generate_signals(self):
        print("[Strategy] Ejecutando señales con estrategia de medias móviles")
        return Signals(buy=[1, 0], sell=[0, 1], hold=[0, 0])

class RSIStrategy(CandlestickStrategy):
    def generate_signals(self):
        print("[Strategy] Ejecutando señales con estrategia basada en RSI")
        return Signals(buy=[1, 0], sell=[0, 1], hold=[0, 0])

# Contexto
class BacktestEngine:
    def __init__(self, strategy: TradingStrategy):
        self.strategy = strategy

    def set_strategy(self, strategy: TradingStrategy):
        print(f"[BacktestEngine] Cambiando estrategia a: {strategy.__class__.__name__}")
        self.strategy = strategy

    def run_backtest(self):
        print("[BacktestEngine] Iniciando backtest")
        return print(self.strategy.generate_signals())

if __name__ == '__main__':

    # Crear algunas velas de ejemplo
    candlesticks = [
        Candlestick(open=[1.0], high=[1.2], low=[0.8], close=[1.1]),
        Candlestick(open=[1.1], high=[1.3], low=[0.9], close=[1.2])
    ]

    # Crear el motor de backtest con una estrategia inicial
    ma_strategy = MovingAverageStrategy(candlesticks=candlesticks)
    engine = BacktestEngine(ma_strategy)
    
    # Ejecutar el backtest con la estrategia inicial
    engine.run_backtest()
    
    # Cambiar a otra estrategia y ejecutar de nuevo
    rsi_strategy = RSIStrategy(candlesticks=candlesticks)
    engine.set_strategy(RSIStrategy())
    engine.run_backtest()

[BacktestEngine] Iniciando backtest
[Strategy] Ejecutando señales con estrategia de medias móviles
Signals(buy=[1, 0], sell=[0, 1], hold=[0, 0])
[BacktestEngine] Cambiando estrategia a: RSIStrategy
[BacktestEngine] Iniciando backtest
[Strategy] Ejecutando señales con estrategia basada en RSI
Signals(buy=[1, 0], sell=[0, 1], hold=[0, 0])


## Problema 2: **Patron observer**
En este escenario podemos usar el patron observer para actualizar la data que usa en el BacktestEngine para validar las estrategias de inversion. Así mismo podemos aprovechar la inversion de dependencias con el fin de reducir el acoplamiento entre el DataFeed y los Observers y en caso tal, agregar mas observers.




![ObserverPattern](ObserverPattern.svg)

In [5]:
@dataclass
class MarketData:
    """Clase que representa los datos del mercado."""
    data: List[Candlestick]

    def update_market(self, new_data: Candlestick):
        print(f"[MarketData] Actualizando datos del mercado con nueva vela: {new_data}")
        self.data.append(new_data)
    
    def get_last_candle(self):
        if self.data:
            return self.data[-1]

    def get_last_price(self):
        if self.data:
            return self.data[-1].close[0]
        return None
    


# Interfaz del observador
class Observer(ABC):
    @abstractmethod
    def update(self, price: float):
        pass

# Observadores concretos
class PlotObserver(Observer):
    def update(self, data: MarketData):
        print(f"[PlotObserver] Graficando precio: {data.get_last_price()}")

class LoggerObserver(Observer):
    def update(self, data: MarketData):
        print(f"[LoggerObserver] Registrando precio: {data.get_last_price()}")

class BacktestObserver(Observer):
    """Observador que ejecuta un backtest con los datos del data."""
    def __init__(self, backtest_engine: BacktestEngine):
        self.backtest_engine = backtest_engine

    def update(self, data: MarketData):
        print(f"[BacktestObserver] Ejecutando backtest con precio de cierre: {data.get_last_price()}")
        self.backtest_engine.run_backtest()


# Datos del mercado observados
class MarketDataFeed:
    def __init__(self, market_data: MarketData):
        self.market_data = market_data
        self.observers :  List[Observer] = []

    def attach(self, observer: Observer):
        self.observers.append(observer)

    def detach(self, observer: Observer):
        self.observers.remove(observer)

    def update(self, market_data: MarketData):
        print(f"[MarketDataFeed] Actualizando datos del mercado: {market_data.get_last_candle()}")
        self.market_data = market_data
        
    def notify(self):
        print(f"[MarketDataFeed] Nuevo precio recibido: {self.market_data.get_last_price()}")
        for obs in self.observers:
            obs.update(self.market_data)

if __name__ == "__main__":
    # Crear datos del mercado
    market_data = MarketData(data=candlesticks)

    # Crear el feed de datos del mercado
    market_feed = MarketDataFeed(market_data)

    # Crear observadores
    plot_observer = PlotObserver()
    logger_observer = LoggerObserver()
    backtest_engine = BacktestEngine(ma_strategy)
    backtest_observer = BacktestObserver(backtest_engine)

    

    # Adjuntar observadores al feed de datos del mercado
    market_feed.attach(plot_observer)
    market_feed.attach(logger_observer)
    market_feed.attach(backtest_observer)

    market_feed.notify()

    print('\n[Main] Actualizando siguiente vela:')
    # Actualizar el mercado con una nueva vela
    new_candle = Candlestick(open=[1.2], high=[1.4], low=[1.0], close=[1.1])
    market_data.update_market(new_candle)
    market_feed.update(market_data)
    
    # Notificar a los observadores sobre el nuevo precio
    market_feed.notify()



[MarketDataFeed] Nuevo precio recibido: 1.2
[PlotObserver] Graficando precio: 1.2
[LoggerObserver] Registrando precio: 1.2
[BacktestObserver] Ejecutando backtest con precio de cierre: 1.2
[BacktestEngine] Iniciando backtest
[Strategy] Ejecutando señales con estrategia de medias móviles
Signals(buy=[1, 0], sell=[0, 1], hold=[0, 0])

[Main] Actualizando siguiente vela:
[MarketData] Actualizando datos del mercado con nueva vela: Candlestick(open=[1.2], high=[1.4], low=[1.0], close=[1.1])
[MarketDataFeed] Actualizando datos del mercado: Candlestick(open=[1.2], high=[1.4], low=[1.0], close=[1.1])
[MarketDataFeed] Nuevo precio recibido: 1.1
[PlotObserver] Graficando precio: 1.1
[LoggerObserver] Registrando precio: 1.1
[BacktestObserver] Ejecutando backtest con precio de cierre: 1.1
[BacktestEngine] Iniciando backtest
[Strategy] Ejecutando señales con estrategia de medias móviles
Signals(buy=[1, 0], sell=[0, 1], hold=[0, 0])


## Problema 3: **Template method**
A partir del template method podemos definir una plantilla para estructurar la tipología del backtest engine que queremos implementar en nuestro proyecto por ejemplo diseñar un backtest que tome los datos a partir de archivos planos, otro encargado de descargar la data de una API, o tambien un para realizar el backtest en vivo a partir de websockets.




![TemplateMethodPattern](TemplateMethodPattern.svg)

In [17]:
from abc import ABC, abstractmethod

class WebClient(ABC):
    """Clase abstracta que define la interfaz para un cliente web."""

    @abstractmethod
    def connect(self, url: str):
        """Conecta al servidor."""
        pass

    @abstractmethod
    def send(self, message: str):
        """Envía un mensaje al servidor."""
        pass

    @abstractmethod
    def receive(self) -> str:
        """Recibe un mensaje del servidor."""
        pass

    @abstractmethod
    def close(self):
        """Cierra la conexión."""
        pass

class WebsocketClient(WebClient):
    """Cliente que utiliza WebSockets para comunicarse con el servidor."""

    def connect(self, url: str):
        print(f"[WebsocketClient] Conectando a {url}")

    def send(self, message: str):
        print(f"[WebsocketClient] Enviando mensaje: {message}")

    def receive(self) -> str:
        print("[WebsocketClient] Recibiendo mensaje")
        return "Mensaje recibido"

    def close(self):
        print("[WebsocketClient] Cerrando conexión")

class WebApiClient(WebClient):
    """Cliente que utiliza una API REST para comunicarse con el servidor."""

    def connect(self, url: str):
        print(f"[WebApiClient] Conectando a {url}")

    def send(self, message: str):
        print(f"[WebApiClient] Enviando mensaje: {message}")

    def receive(self) -> str:
        print("[WebApiClient] Recibiendo mensaje")
        return "Respuesta de la API"

    def close(self):
        print("[WebApiClient] Cerrando conexión")

class BacktestEngine:
    """Clase abstracta que define la interfaz para un motor de backtest."""

    @abstractmethod
    def set_file(self, data: str):
        """Establece los archivos de mercado para el backtest."""
        pass

    @abstractmethod
    def read_file(self, file_path: str):
        """Lee los datos del archivo de mercado."""
        pass

    @abstractmethod
    def set_client(self, client: WebClient):
        """Establece los datos del mercado para el backtest."""
        pass

    @abstractmethod
    def connect_market(self):
        """Conecta el motor de backtest con los datos del mercado."""
        pass

    @abstractmethod
    def set_strategy(self, strategy: TradingStrategy):
        """Establece la estrategia de trading a utilizar en el backtest."""
        pass

    @abstractmethod
    def run_live_backtest(self):
        """Ejecuta el backtest en tiempo real utilizando la estrategia establecida."""
        pass

    @abstractmethod
    def run_backtest(self):
        """Ejecuta el backtest utilizando la estrategia establecida."""
        pass

    def run(self):
        """Template method para ejecutar el flujo completo del backtest."""
        print(f"[{self.__class__.__name__}] Iniciando ejecución del backtest")

        try:
            if hasattr(self, "read_file") and type(self).read_file is not BacktestEngine.read_file:
                print(f"[{self.__class__.__name__}] Ejecutando read_file()")
                self.read_file()

            if hasattr(self, "connect_market") and type(self).connect_market is not BacktestEngine.connect_market:
                print(f"[{self.__class__.__name__}] Ejecutando connect_market()")
                self.connect_market()

            if hasattr(self, "run_live_backtest") and type(self).run_live_backtest is not BacktestEngine.run_live_backtest:
                print(f"[{self.__class__.__name__}] Ejecutando backtest en vivo")
                self.run_live_backtest()

            elif hasattr(self, "run_backtest") and type(self).run_backtest is not BacktestEngine.run_backtest:
                print(f"[{self.__class__.__name__}] Ejecutando backtest offline")
                self.run_backtest()

            else:
                print(f"[{self.__class__.__name__}] Ningún método de ejecución implementado")

        except NotImplementedError as nie:
            print(f"[{self.__class__.__name__}] Método no implementado: {nie}")
        except Exception as e:
            print(f"[{self.__class__.__name__}] Error durante la ejecución: {e}")
        

class CsvBacktestEngine(BacktestEngine):
    def __init__(self):
        self.file_path : str = None
        self.data: MarketData = None
        self.strategy: TradingStrategy = None

    def set_file(self, file_path: str):
        print(f"[CsvBacktestEngine] Estableciendo archivo de mercado: {file_path}")
        self.file_path = file_path
    
    def read_file(self):
        if not self.file_path:
            print("[CsvBacktestEngine] No se ha establecido un archivo de mercado.")
            return
        print(f"[CsvBacktestEngine] Leyendo datos del archivo: {self.file_path}")
        # Aquí se simula la lectura de un archivo CSV y la creación de MarketData
        self.data = MarketData(data=[
            Candlestick(open=[1.0], high=[1.2], low=[0.8], close=[1.1]),
            Candlestick(open=[1.1], high=[1.3], low=[0.9], close=[1.2])
        ])

    def set_strategy(self, strategy: TradingStrategy):
        print(f"[CsvBacktestEngine] Estableciendo estrategia: {strategy.__class__.__name__}")
        self.strategy = strategy
    
    def run_backtest(self):
        if not self.strategy:
            print("[CsvBacktestEngine] No se ha establecido una estrategia.")
            return
        print("[CsvBacktestEngine] Ejecutando backtest")
        signals = self.strategy.generate_signals()
        print(f"[CsvBacktestEngine] Señales generadas: {signals}")


class ApiBacktestEngine(BacktestEngine):
    def __init__(self):
        self.market_data: WebClient = None
        self.strategy: TradingStrategy = None

    def set_client(self, market_data: WebClient):
        print(f"[ApiBacktestEngine] Conectando al mercado con {market_data.__class__.__name__}")
        self.market_data = market_data

    def connect_market(self):
        if not self.market_data:
            print("[ApiBacktestEngine] No se ha establecido un cliente de mercado.")
            return
        print(f"[ApiBacktestEngine] Conectando al mercado con {self.market_data.__class__.__name__}")

    def set_strategy(self, strategy: TradingStrategy):
        print(f"[ApiBacktestEngine] Estableciendo estrategia: {strategy.__class__.__name__}")
        self.strategy = strategy
    
    def run_backtest(self):
        if not self.strategy:
            print("[ApiBacktestEngine] No se ha establecido una estrategia.")
            return
        print("[ApiBacktestEngine] Ejecutando backtest")
        signals = self.strategy.generate_signals()
        print(f"[ApiBacktestEngine] Señales generadas: {signals}")


class WebsocketBacktestEngine(BacktestEngine):
    def __init__(self):
        self.market_data: WebClient = None
        self.strategy: TradingStrategy = None

    def set_client(self, market_data: WebClient):
        print(f"[WebsocketBacktestEngine] Conectando al mercado con {market_data.__class__.__name__}")
        self.market_data = market_data

    def connect_market(self):
        if not self.market_data:
            print("[WebsocketBacktestEngine] No se ha establecido un cliente de mercado.")
            return
        print(f"[WebsocketBacktestEngine] Conectando al mercado con {self.market_data.__class__.__name__}")

    def set_strategy(self, strategy: TradingStrategy):
        print(f"[WebsocketBacktestEngine] Estableciendo estrategia: {strategy.__class__.__name__}")
        self.strategy = strategy
    
    def run_live_backtest(self):
        if not self.strategy:
            print("[WebsocketBacktestEngine] No se ha establecido una estrategia.")
            return
        print("[WebsocketBacktestEngine] Ejecutando backtest")
        signals = self.strategy.generate_signals()
        print(f"[WebsocketBacktestEngine] Señales generadas: {signals}")


if __name__=='__main__':
    
    # Crear el motor de backtest con una estrategia inicial
    ma_strategy = MovingAverageStrategy(candlesticks=candlesticks)
    csv_engine = CsvBacktestEngine()
    csv_engine.set_file("market_data.csv")
    csv_engine.set_strategy(ma_strategy)
    csv_engine.run()
    print("\n")
    
    # Crear un motor de backtest para API
    api_engine = ApiBacktestEngine()
    api_engine.set_client(WebApiClient())
    api_engine.set_strategy(RSIStrategy())
    api_engine.run()
    print("\n")

    # Crear un motor de backtest para WebSocket
    websocket_engine = WebsocketBacktestEngine()
    websocket_engine.set_client(WebsocketClient())
    websocket_engine.set_strategy(MovingAverageStrategy())
    websocket_engine.run()

[CsvBacktestEngine] Estableciendo archivo de mercado: market_data.csv
[CsvBacktestEngine] Estableciendo estrategia: MovingAverageStrategy
[CsvBacktestEngine] Iniciando ejecución del backtest
[CsvBacktestEngine] Ejecutando read_file()
[CsvBacktestEngine] Leyendo datos del archivo: market_data.csv
[CsvBacktestEngine] Ejecutando backtest offline
[CsvBacktestEngine] Ejecutando backtest
[Strategy] Ejecutando señales con estrategia de medias móviles
[CsvBacktestEngine] Señales generadas: Signals(buy=[1, 0], sell=[0, 1], hold=[0, 0])


[ApiBacktestEngine] Conectando al mercado con WebApiClient
[ApiBacktestEngine] Estableciendo estrategia: RSIStrategy
[ApiBacktestEngine] Iniciando ejecución del backtest
[ApiBacktestEngine] Ejecutando connect_market()
[ApiBacktestEngine] Conectando al mercado con WebApiClient
[ApiBacktestEngine] Ejecutando backtest offline
[ApiBacktestEngine] Ejecutando backtest
[Strategy] Ejecutando señales con estrategia basada en RSI
[ApiBacktestEngine] Señales generadas: Si